In [19]:
import json
import addressinfo_craw as addrc


def load_addrinfo(addrinfo_list):
    """
    把处理好的地址信息列表存入本地addressinfo.json文件
    """
    with open('addressinfo.json','a') as f:
        for line in addrinfo_list:
            f.write(json.dumps(line,ensure_ascii=False,indent=2 )) 
            f.write(',')

            
def get_addrsinfo(url,addinfo_list):
    """
    格式化好之后的每个地址信息单独成为了一个文件，
    需把65个地址放入一个列表中
    """   
    with open(url,'r') as f:
        str_json = json.loads(f.read())
#         print(str_json['addrs'])
        for attr in str_json['addrs']:
            addinfo_list.append(attr)


def main():
    address_list = addrc.get_addrslist()
    print(len(address_list))
    addrinfo_list = []

    while len(address_list)>0:
        address = address_list.pop()
        if len(address) == 34:
            url = 'addrsdata/' + address + '.json'
            get_addrsinfo(url,addrinfo_list)
        else:
            continue
   
    print("总共地址信息数据数量：",len(addrinfo_list))
    load_addrinfo(addrinfo_list)


69
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
